### Light GBM
##### XGBoost'un eğitim süresi performansını arttırmaya yönelik geliştirilen bir diğer GBM türüdür.
###### Daha performanslı
###### Level-wise büyüme stratejisi yerine Leaf-wise büyüme stratejisi kullanır.
###### Breadth-first search(BFS) yerine depth-first search (DFS) kullanır.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [2]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])
y = df["Salary"]
X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis=1).astype("float64")
X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

#### Model & Prediction

In [5]:
!pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 300.6 kB/s eta 0:00:00


In [6]:
from lightgbm import LGBMRegressor

In [9]:
lgb_model = LGBMRegressor().fit(X_train, y_train)

In [10]:
y_pred = lgb_model.predict(X_test)

In [11]:
np.sqrt(mean_squared_error(y_test, y_pred))

363.8712087611089

#### Model Tuning

In [16]:
lgb_params = {"learning_rate": [0.01, 0.1, 0.5, 1],
              "n_estimators": [20, 40, 100, 200, 500, 1000],
              "max_depth": [1,2,3,4,5,6,7,8,9,10]}

In [17]:
lgb_cv_model = GridSearchCV(lgb_model, lgb_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


In [18]:
lgb_cv_model.best_params_

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 20}

In [19]:
lgb_tuned = LGBMRegressor(learning_rate=0.1, max_depth=6, n_estimators=20).fit(X_train, y_train)

In [20]:
y_pred = lgb_tuned.predict(X_test)

In [21]:
np.sqrt(mean_squared_error(y_test, y_pred))

371.5044868943621